In [92]:
import numpy as np
import pandas as pd
import json
from pathlib import Path
from typing import Any
import joblib
import numpy as np
import numpy.typing as npt
from sklearn.tree import DecisionTreeRegressor

In [93]:
data = pd.read_csv("kc_house_data.csv")

In [94]:
data

id             date     price  bedrooms  bathrooms  \
0      7129300520  20141013T000000  221900.0         3       1.00   
1      6414100192  20141209T000000  538000.0         3       2.25   
2      5631500400  20150225T000000  180000.0         2       1.00   
3      2487200875  20141209T000000  604000.0         4       3.00   
4      1954400510  20150218T000000  510000.0         3       2.00   
...           ...              ...       ...       ...        ...   
21608   263000018  20140521T000000  360000.0         3       2.50   
21609  6600060120  20150223T000000  400000.0         4       2.50   
21610  1523300141  20140623T000000  402101.0         2       0.75   
21611   291310100  20150116T000000  400000.0         3       2.50   
21612  1523300157  20141015T000000  325000.0         2       0.75   

       sqft_living  sqft_lot  floors  waterfront  view  ...  grade  \
0             1180      5650     1.0           0     0  ...      7   
1             2570      7242     2.0           0     0  ...      7   
2              770     10000     1.0           0     0  ...      6   
3             1960      5000     1.0           0     0  ...      7   
4             1680      8080     1.0           0     0  ...      8   
...            ...       ...     ...         ...   ...  ...    ...   
21608         1530      1131     3.0           0     0  ...      8   
21609         2310      5813     2.0           0     0  ...      8   
21610         1020      1350     2.0           0     0  ...      7   
21611         1600      2388     2.0           0     0  ...      8   
21612         1020      1076     2.0           0     0  ...      7   

       sqft_above  sqft_basement  yr_built  yr_renovated  zipcode      lat  \
0            1180              0      1955             0    98178  47.5112   
1            2170            400      1951          1991    98125  47.7210   
2             770              0      1933             0    98028  47.7379   
3            1050            910      1965             0    98136  47.5208   
4            1680              0      1987             0    98074  47.6168   
...           ...            ...       ...           ...      ...      ...   
21608        1530              0      2009             0    98103  47.6993   
21609        2310              0      2014             0    98146  47.5107   
21610        1020              0      2009             0    98144  47.5944   
21611        1600              0      2004             0    98027  47.5345   
21612        1020              0      2008             0    98144  47.5941   

          long  sqft_living15  sqft_lot15  
0     -122.257           1340        5650  
1     -122.319           1690        7639  
2     -122.233           2720        8062  
3     -122.393           1360        5000  
4     -122.045           1800        7503  
...        ...            ...         ...  
21608 -122.346           1530        1509  
21609 -122.362           1830        7200  
21610 -122.299           1020        2007  
21611 -122.069           1410        1287  
21612 -122.299           1020        1357  

[21613 rows x 21 columns]

In [95]:
print(len(data["id"].unique()))

21436


In [96]:
data.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [97]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import copy
data_copy = data.copy()
del data_copy["id"], data_copy["date"], data_copy["zipcode"]

X = data_copy[list(set(data_copy.columns) - set(["price"]))]
y = data_copy["price"].apply(np.log) * 100

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train, y_train)
X_scaled_test = scaler.transform(X_test)

In [98]:
print(X_scaled_train, type(X_scaled_train), X_scaled_train.shape)
print(X_scaled_test, type(X_scaled_test), X_scaled_test.shape)

print(y_train, type(y_train), y_train.shape)
print(y_test, type(y_test), y_test.shape)

[[ 0.9317943  -0.36292767 -0.64422392 ... -0.41085449 -1.55387844
  -0.02444164]
 [-0.91834232 -0.41013413 -0.01105006 ... -0.41085449 -1.81873016
  -1.15339258]
 [-0.91834232 -1.24532541 -1.30791218 ... -1.50480339  0.02729354
  -0.43626022]
 ...
 [-0.91834232 -0.70063545 -0.05464207 ... -0.41085449 -0.13868982
  -0.60666791]
 [-0.91834232 -0.94271988 -1.16623817 ... -0.41085449 -0.79973662
   0.26667149]
 [-0.91834232 -0.26488347 -0.55595012 ... -0.41085449  1.49949384
  -1.03268713]] <class 'numpy.ndarray'> (15129, 17)
[[ 0.9317943   1.64758352  1.16593404 ...  0.6830944  -0.32199319
   0.08206316]
 [-0.91834232 -1.07586631 -0.85019614 ... -0.41085449 -0.16683483
  -0.40075862]
 [ 0.9317943   0.24349383 -0.09823407 ... -0.41085449  1.25701379
  -0.08834452]
 ...
 [-0.91834232 -1.05165787 -0.61044012 ... -1.50480339  0.85937538
  -0.4646615 ]
 [ 0.9317943   0.32822338 -0.02194807 ... -0.41085449 -1.37057507
   1.38852211]
 [-0.91834232 -0.76115655  0.26139996 ...  0.6830944  -0.37106

In [99]:
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
print(y_train, type(y_train), y_train.shape)
print(y_test, type(y_test), y_test.shape)

[1245.25420221 1244.5089546  1283.73444224 ... 1279.38593104 1214.9502294
 1279.38593104] <class 'numpy.ndarray'> (15129,)
[1343.23684278 1266.03279178 1301.70028617 ... 1333.73941086 1273.66685431
 1293.8680895 ] <class 'numpy.ndarray'> (6484,)


In [100]:
from random_forest import RandomForestMSE
import time

grid_dict = {"max_depth": [None, 10,20], "max_features": [1,3,5]}
rmsle_history = []
time_history = []
estimators = []
depths = []
Fdimensions = []

for n_est in [10,20,30]:
  for mx_dpth in grid_dict["max_depth"]:
    for mx_feat in grid_dict["max_features"]:
      RF = RandomForestMSE(n_estimators = n_est, tree_params = {"max_depth": mx_dpth, "max_features": mx_feat})
      start = time.time()
      hist = RF.fit(X = X_scaled_train, y = y_train, X_val = X_scaled_test, y_val = y_test, patience = 2)
      y_pred = RF.predict(X_scaled_test)
      end = time.time() - start
      estimators.append(n_est)
      depths.append(mx_dpth)
      Fdimensions.append(mx_feat)
      rmsle_history.append(hist)
      time_history.append(end)

In [106]:
RFres = pd.read_csv('RF.csv')
del RFres["Unnamed: 0"]
RFres

time                                  rmsle_convergence  estimators  \
0    1.674777  {'train': [np.float64(0.039952363273255846), n...          10   
1    1.864074  {'train': [np.float64(0.039955365778136), np.f...          10   
2    2.222877  {'train': [np.float64(0.03994946924942101), np...          10   
3    1.732726  {'train': [np.float64(0.039960144566811774), n...          10   
4    1.349464  {'train': [np.float64(0.03995917397093531), np...          10   
5    1.574978  {'train': [np.float64(0.03994764874759863), np...          10   
6    1.441680  {'train': [np.float64(0.03994793831004262), np...          10   
7    1.742297  {'train': [np.float64(0.03997061251926755), np...          10   
8    2.286336  {'train': [np.float64(0.03995031867476678), np...          10   
9    5.697107  {'train': [np.float64(0.039944719324806234), n...          20   
10   5.526019  {'train': [np.float64(0.039954837528044615), n...          20   
11   6.022363  {'train': [np.float64(0.03995018028633169), np...          20   
12   3.228732  {'train': [np.float64(0.039949227700433694), n...          20   
13   4.956249  {'train': [np.float64(0.039948180380602015), n...          20   
14   5.139208  {'train': [np.float64(0.0399560531933967), np....          20   
15   5.987273  {'train': [np.float64(0.03996550065050771), np...          20   
16   5.495207  {'train': [np.float64(0.039953850263734665), n...          20   
17   6.992310  {'train': [np.float64(0.039945392565006184), n...          20   
18  10.384207  {'train': [np.float64(0.03994941123386166), np...          30   
19  12.306204  {'train': [np.float64(0.03995038520683968), np...          30   
20  12.574167  {'train': [np.float64(0.03995651829315267), np...          30   
21   6.900092  {'train': [np.float64(0.039962625006963196), n...          30   
22   8.072668  {'train': [np.float64(0.03994389821664878), np...          30   
23   7.810603  {'train': [np.float64(0.03995731920186512), np...          30   
24   9.081037  {'train': [np.float64(0.03994900887472613), np...          30   
25  12.575536  {'train': [np.float64(0.039949360445914496), n...          30   
26  13.211730  {'train': [np.float64(0.03995391494505471), np...          30   

    depth  features_per_leaf  
0     NaN                  1  
1     NaN                  3  
2     NaN                  5  
3    10.0                  1  
4    10.0                  3  
5    10.0                  5  
6    20.0                  1  
7    20.0                  3  
8    20.0                  5  
9     NaN                  1  
10    NaN                  3  
11    NaN                  5  
12   10.0                  1  
13   10.0                  3  
14   10.0                  5  
15   20.0                  1  
16   20.0                  3  
17   20.0                  5  
18    NaN                  1  
19    NaN                  3  
20    NaN                  5  
21   10.0                  1  
22   10.0                  3  
23   10.0                  5  
24   20.0                  1  
25   20.0                  3  
26   20.0                  5

In [108]:
from last_boosting import GradientBoostingMSE
import time

grid_dict = {"max_depth": [None, 10,20], "max_features": [1,3,5]}
lr_list = [1e-3, 0.01, 0.1]
boosting_rmsle_history = []
boosting_time_history = []
estimators = []
depths = []
Fdimensions = []
learning_rates = []

for n_est in [10,20,30]:
  for mx_dpth in grid_dict["max_depth"]:
    for mx_feat in grid_dict["max_features"]:
      for lr in lr_list:
        GB = GradientBoostingMSE(n_estimators = n_est, tree_params = {"max_depth": mx_dpth, "max_features": mx_feat}, learning_rate = lr)
        start = time.time()
        hist = GB.fit(X = X_scaled_train, y = y_train, X_val = X_scaled_test, y_val = y_test, patience = 2)
        y_pred = GB.predict(X_scaled_test)
        end = time.time() - start
        boosting_rmsle_history.append(hist)
        boosting_time_history.append(end)
        estimators.append(n_est)
        depths.append(mx_dpth)
        Fdimensions.append(mx_feat)
        learning_rates.append(lr)

In [110]:
GBres = pd.read_csv('GB.csv')
del GBres["Unnamed: 0"]
GBres

time                                  rmsle_convergence  estimators  \
0    2.176996  {'train': [np.float64(0.03995184107218973), np...          10   
1    1.629280  {'train': [np.float64(0.03995184107218973), np...          10   
2    1.546616  {'train': [np.float64(0.03995184107218973), np...          10   
3    2.761162  {'train': [np.float64(0.03995184107218973), np...          10   
4    2.278963  {'train': [np.float64(0.03995184107218973), np...          10   
..        ...                                                ...         ...   
76  15.112815  {'train': [np.float64(0.03995184107218973), np...          30   
77  13.753502  {'train': [np.float64(0.03995184107218973), np...          30   
78  17.791708  {'train': [np.float64(0.03995184107218973), np...          30   
79  13.654414  {'train': [np.float64(0.03995184107218973), np...          30   
80  22.729496  {'train': [np.float64(0.03995184107218973), np...          30   

    depth  features_per_leaf  learning_rate  
0     NaN                  1          0.001  
1     NaN                  1          0.010  
2     NaN                  1          0.100  
3     NaN                  3          0.001  
4     NaN                  3          0.010  
..    ...                ...            ...  
76   20.0                  3          0.010  
77   20.0                  3          0.100  
78   20.0                  5          0.001  
79   20.0                  5          0.010  
80   20.0                  5          0.100  

[81 rows x 6 columns]